# Bayesian Structural Time Series Model

In [1]:
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
import tensorflow_probability as tfp

from tensorflow_probability import distributions as tfd
from tensorflow_probability import sts

KeyboardInterrupt: 

In [ ]:
sns.set(rc={'figure.figsize': (12,8)})

In [ ]:
date_parser = lambda x: pd.datetime.strptime(x, "%d/%m/%Y 12:00:00 a.m.")
df = pd.read_csv('data/datasetRofex2.csv',
                 parse_dates=['Fecha'],
                 index_col='Fecha',
                 date_parser=date_parser)

## Model

In [ ]:
tf.reset_default_graph()

trend = sts.LocalLinearTrend(observed_time_series=df['Cierre'])
seasonal = tfp.sts.Seasonal(num_seasons=33,
                            num_steps_per_season=123,
                            observed_time_series=df['Cierre'])
model = sts.Sum([trend, seasonal], observed_time_series=df['Cierre'])

with tf.variable_scope('sts_elbo', reuse=tf.AUTO_REUSE):
    elbo_loss, variational_posteriors = tfp.sts.build_factored_variational_loss(
        model, observed_time_series=df['Cierre'])

In [ ]:
# Minimize the variational loss.

steps = 300

train_vi = tf.train.AdamOptimizer(0.1).minimize(elbo_loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(steps):
        _, elbo_ = sess.run((train_vi, elbo_loss))
        if i % 20 == 0:
            print("step {} -ELBO {}".format(i, elbo_))

        samples = sess.run({k: q.sample(50)
                                    for k, q in variational_posteriors.items()})

In [ ]:
for param in model.parameters:
    print("{}: {} +- {}".format(param.name,
                                np.mean(samples[param.name], axis=0),
                                np.std(samples[param.name], axis=0)))

In [ ]:
price_dist = tfp.sts.forecast(model,
                              observed_time_series=df['Cierre'],
                              parameter_samples=samples,
                              num_steps_forecast=steps)

In [ ]:
num_samples = 10

with tf.Session() as sess:
    price_mean, price_scale, price_forecast_samples = sess.run(
        (price_dist.mean()[..., 0], price_dist.stddev()[..., 0],
         price_dist.sample(num_samples)[..., 0]))

In [ ]:
last_day = df.index[-1]
forecast_period = pd.date_range(start=last_day + pd.DateOffset(1), periods=steps, freq='B')

In [ ]:
forecasts = pd.DataFrame({'forecast': price_mean}, index=forecast_period)
forecast_df = df.append(forecasts, sort=True)

In [ ]:
colors = sns.color_palette()
c1, c2 = colors[0], colors[1]

ax = sns.lineplot(x=forecast_df.index, y=forecast_df['Cierre'])
sns.lineplot(x=forecast_df.index, y=forecast_df['forecast'], ax=ax)
ax.plot(forecast_period, price_forecast_samples.T, lw=1, color=c2, alpha=0.1);
ax.fill_between(forecast_period,
                price_mean - 2 * price_scale,
                price_mean + 2 * price_scale,
                color=c2,
                alpha=0.2);